In [1]:
import numpy as np
import pandas as pd
from pandas_datareader import data as web
import matplotlib.pyplot as plt

/home/itamar/anaconda3/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [6]:
tickers = ['LOGN3.SA','MAERSK-B.CO']

In [7]:
sec_data = pd.DataFrame()
for t in tickers:
    sec_data[t] = web.DataReader(t,data_source='yahoo',start ='2010-1-1')['Adj Close']

In [13]:
sec_data.tail()

,LOGN3.SA,MAERSK-B.CO
Date,,
2020-05-25,13.80,6942.0
2020-05-26,13.47,6986.0
2020-05-27,14.18,6752.0
2020-05-28,14.63,6580.0
2020-05-29,13.95,6550.0


In [16]:
sec_returns = np.log(sec_data/sec_data.shift(1))
sec_returns.tail()

,LOGN3.SA,MAERSK-B.CO
Date,,
2020-05-25,0.110203,NaN
2020-05-26,-0.024204,0.006318
2020-05-27,0.051368,-0.034069
2020-05-28,0.031242,-0.025804
2020-05-29,-0.047595,-0.004570


## LOGN3

In [17]:
sec_returns['LOGN3.SA'].mean()

-0.0003706832519844604

In [18]:
sec_returns['LOGN3.SA'].mean()*250

-0.0926708129961151

In [19]:
sec_returns['LOGN3.SA'].std()

0.03777511802099218

In [20]:
sec_returns['LOGN3.SA'].std() * 250 ** 0.5

0.5972770591400376

## MAERSK

In [21]:
sec_returns['MAERSK-B.CO'].mean()

0.000395468247316074

In [22]:
sec_returns['MAERSK-B.CO'].mean()*250

0.0988670618290185

In [23]:
sec_returns['MAERSK-B.CO'].std()

0.021712308961954956

In [24]:
sec_returns['MAERSK-B.CO'].std() * 250 ** 0.5

0.34330174790531925

## COVARIÂNCIA
$$
\sigma_{xy} = \frac{(x - \bar x) * (y - \bar y)}{n - 1}
$$

<ul>
    <li>covar > 0 --> Move in same direction</li>
    <li>covar &lt; 0 --> Move in opposite direction</li>
    <li>covar = 0 --> Independent variables </li>
</ul>

$$
CovarianceMatrix: \sum = 
 \begin{bmatrix}
       \sigma_{1^2} & \sigma_{12} & \dots & \sigma_{1t}           \\[0.3em]
       \sigma_{21} & \sigma_{2^2} & \dots & \sigma_{2t}  \\[0.3em]
       \vdots & \vdots & \ddots & \vdots \\[0.3em]
       \sigma_{t1} & \sigma_{t2} & \dots & \sigma_{t^2}
 \end{bmatrix}
$$

In [26]:
logn_var = sec_returns['LOGN3.SA'].var()
logn_var

0.001426959541499888

In [27]:
maersk_var = sec_returns['MAERSK-B.CO'].var()
maersk_var

0.00047142436045938946

In [29]:
cov_matrix = sec_returns.cov()
cov_matrix

,LOGN3.SA,MAERSK-B.CO
LOGN3.SA,0.001427,0.000135
MAERSK-B.CO,0.000135,0.000471


In [31]:
cov_matrix_a = sec_returns.cov() * 250
cov_matrix_a

,LOGN3.SA,MAERSK-B.CO
LOGN3.SA,0.356740,0.033864
MAERSK-B.CO,0.033864,0.117856


## Correlação
$$
corr = \frac{Cov(X,Y)}{S(X) * S(Y)}
$$
* S -> std

In [32]:
corr_matrix = sec_returns.corr()
corr_matrix

,LOGN3.SA,MAERSK-B.CO
LOGN3.SA,1.000000,0.165267
MAERSK-B.CO,0.165267,1.000000


## Portifolio Risk
1. Equal weight

In [33]:
weights = np.array([0.5,0.5])

Portifolio Variance

In [58]:
portifolio_var = np.dot(weights.T, np.dot(sec_returns.cov() * 250, weights))

In [59]:
portifolio_var

0.13558114324980544

Portifolio Volatility

In [39]:
portifolio_vol = portifolio_var ** 0.5
portifolio_vol

0.36821344794806915

## Calculating Diversifiable and non-diversifiable portifolio risk

#### Diversifiable

In [60]:
logn_var_a = logn_var* 250
maersk_var_a = maersk_var * 250

In [65]:
diversifiable_risk = portifolio_var - ((weights[0] ** 2 * logn_var_a) - (weights[1] ** 2 * maersk_var_a))
diversifiable_risk

0.07586019443477429

In [66]:
print(str(round(diversifiable_risk*100,3))+ ' %')

7.586 %


#### Non diversifiable

In [67]:
n_dr_1 = portifolio_var - diversifiable_risk
n_dr_1

0.05972094881503115

In [68]:
n_dr_2 = (weights[0] ** 2 * logn_var_a) - (weights[1] ** 2 * maersk_var_a)
n_dr_2

0.05972094881503115

In [69]:
assert(n_dr_1 == n_dr_2)